# CS246 - Colab 5
## PageRank

### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [5]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [6]:
nb_nodes = G.number_of_nodes()
nb_edges = G.number_of_edges()
print(f'Graph G with {nb_nodes} nodes and {nb_edges} edges')

Graph G with 281903 nodes and 2312497 edges


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

In [9]:
# YOUR CODE HERE
lst_max_weakly_connected_components = max(nx.weakly_connected_components(G))
len(lst_max_weakly_connected_components)

255265

In [10]:
G.remove_nodes_from(set(G.nodes()) - lst_max_weakly_connected_components)

In [11]:
print(nx.info(G))

DiGraph with 255265 nodes and 2234572 edges


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [12]:
# YOUR CODE HERE
pageRank = nx.pagerank(G)

In [13]:
len(pageRank)

255265

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using for both methods ```seed = 1```, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.


In [14]:
# YOUR CODE HERE
nb_nodes = G.number_of_nodes()
nb_edges = G.number_of_edges()

randomGraph = nx.fast_gnp_random_graph(n=nb_nodes, p = 0.00008, seed=1)
barabasiAlbertGraph = nx.barabasi_albert_graph(n=nb_nodes, m=nb_edges// nb_nodes+1, seed=1)

In [15]:
print(nx.info(G))

DiGraph with 255265 nodes and 2234572 edges


In [16]:
print(nx.info(randomGraph))

Graph with 255265 nodes and 2606386 edges


In [17]:
print(nx.info(barabasiAlbertGraph))

Graph with 255265 nodes and 2297304 edges


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure. 

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [19]:
# YOUR CODE HERE
pageRankRandomGraph = nx.pagerank(randomGraph)
len(pageRankRandomGraph)

255265

In [20]:
pageRankBarabasiAlbertGraph = nx.pagerank(barabasiAlbertGraph)
len(pageRankBarabasiAlbertGraph)

255265

In [28]:
sortedPageRank = np.array(sorted(pageRank.values()))
sortedPageRankRandomGraph = np.array(sorted(pageRankRandomGraph.values()))
sortedPageRankBarabasiAlbertGraph = np.array(sorted(pageRankBarabasiAlbertGraph.values()))

In [32]:
from scipy import spatial


def cosine_distance(a, b):
    cosine_distance = float(spatial.distance.cosine(a, b))
    return cosine_distance

print(cosine_distance(sortedPageRank, sortedPageRankRandomGraph))
print(cosine_distance(sortedPageRank, sortedPageRankBarabasiAlbertGraph))

0.896044352960356
0.3511326339032764


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!